In [ ]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# Navigators

In [ ]:
''' NAVIGATOR BETWAY '''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

In [ ]:
''' NAVIGATOR 1XBET '''
url_xbet = 'https://ar.1xbet.com/es/live/football'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#xbet = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
xbet = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
xbet.get(url_xbet)
xbet.set_window_size(1920, 1080)

In [ ]:
''' NAVIGATOR BETSSON '''
url_bet = 'https://pba.betsson.bet.ar/apuestas-deportivas/futbol'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bets = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bets = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bets.get(url_bet)
bets.maximize_window()

In [ ]:
''' NAVIGATOR       20BET LIVE '''
url = 'https://20bet.com/es'
options = Options()
options.headless = False #for not opening the navigator

veinte = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
veinte.get(url)
veinte.maximize_window()

notific = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.align-right.secondary.slidedown-button')))
notific.click()

cookie = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="platform"]/div[2]/div[3]/div/button')))
cookie.click()


# Functions

In [ ]:
def betway():
    df_bway = pd.DataFrame(columns =['teams', '1_bway', 'X_bway', '2_bway'])
    df_bway2 = pd.DataFrame(columns=['teams', 'under_bway', 'goals_bway', 'over_bway'])
    for j in tqdm(range(30)):
        for i in range(5):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                
                players_bway = elemento[3]
                home = elemento[7]
                tie  = elemento[8]
                away = elemento[9]

                df_bway.loc[len(df_bway.index)] = [players_bway, home, tie, away]
                #print(players_bway, '|         |', f'j: {j}, i: {i} ')

            except:
                pass
    time.sleep(0.5)
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    dropdown.click()
    time.sleep(1.5)
    TotalGoals = bway.find_element(By.CSS_SELECTOR, '[collectionitem="total-goals"]')
    TotalGoals.click()
    
    time.sleep(1.5)
    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                teams = elemento[3]
                goals = elemento[7].replace(',', '.')
                under = elemento[8].replace(',', '.')
                over  = elemento[10].replace(',', '.')
                df_bway2.loc[len(df_bway2.index)] = [teams, under, goals, over]
                #print(teams, f'||     goals: {goals}, under: {under}, over: {over} ')
            except:
                pass

    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    time.sleep(1.5)
    HomeXaway = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway.csv')
    return df_bway

In [ ]:
betway()

In [ ]:
def One_xbet():
    df_xbet = pd.DataFrame(columns=['teams', '1_xbet', 'X_xbet', '2_xbet', 'over_xbet', 'goals_xbet', 'under_xbet'])
    for k in range(40):
        for j in range(5):
            try:
                elemento =xbet.find_element(By.XPATH, f'/html/body/div[3]/div[1]/div[2]/div/div/div[4]/div/div/div[1]/div/div[{k}]/div[{j}]').text.split('\n')
                teams = elemento[0]+' '+elemento[1]
                if len(elemento)==24:
                    home = elemento[9]
                    tie  = elemento[10]
                    away = elemento[11]

                    over = elemento[15].replace('-', '1')
                    goals = elemento[16].replace('-', '1')
                    under = elemento[17].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under]
                if len(elemento)==26:
                    home = elemento[11]
                    tie  = elemento[12]
                    away = elemento[13]

                    over = elemento[17].replace('-', '1')
                    goals = elemento[18].replace('-', '1')
                    under = elemento[19].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under]

                
                
                #print(elemento[0], elemento[1],'         |', k,'  ', j, '|',len(elemento))
            except:
                pass
    df_xbet = df_xbet[~df_xbet.teams.str.contains('Campeonato')]
    df_xbet.to_csv('CSV\df_xbet.csv')
    return df_xbet

In [ ]:
One_xbet()


betsson 1

In [ ]:
def betsson_live():    
    df_bet = pd.DataFrame(columns=['teams', '1_bets', 'X_bets', '2_bets', 'under_bets', 'goals_bets', 'over_bets'])
    for k in range(30):
        try:
            elemento = bets.find_element(By.CSS_SELECTOR, f'obg-event-row-container[test-id^="f-"]:nth-of-type({k})').text.split('\n')
            #print(elemento[1]+' '+ elemento[2], f'|| len:{len(elemento)}, nth-of-type: {k}')
            
            teams = elemento[1]+' '+elemento[2]
            
            home = elemento[9]
            tie = elemento[11]
            away = elemento[13]
            
            if len(elemento)==23:
                under = elemento[19]
                goals = float("".join(re.findall(r'\d+', elemento[18])))/10
                over  = elemento[17]
            
            if len(elemento)==30:
                under = elemento[18]
                goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                over = elemento[16]

            if len(elemento)==35:
                under = elemento[19]
                goals = float("".join(re.findall(r'\d+', elemento[18])))/10
                over = elemento[17]      

            if len(elemento) == 21:
                under = elemento[18]
                goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                over = elemento[16]     

            df_bet.loc[len(df_bet.index)]  = [teams, home, tie, away, under, goals, over]
        except:
            pass
    df_bet.to_csv('CSV\df_bet_live.csv')
    return df_bet

In [ ]:
betsson_live()

betsson 2

def betsson_live():    
    df_bet = pd.DataFrame(columns=['teams', '1_bets', 'X_bets', '2_bets', 'under_bets', 'goals_bets', 'over_bets'])
    for k in range(30):
        try:
            elemento = elemento = bets.find_element(By.CSS_SELECTOR, f'obg-event-row-container[test-id^="f-"]:nth-of-type({K})').text.split('\n')
            print(k, len(elemento))
            
            teams = elemento[1]+' '+elemento[2]
            
            home = elemento[9]
            tie = elemento[11]
            away = elemento[13]

            if len(elemento)==26:
                under = elemento[18]
                goals = "".join(re.findall(r'\d+', elemento[17]))
                over = elemento[16]

            if len(elemento)==35:
                under = elemento[19]
                goals = "".join(re.findall(r'\d+', elemento[18]))
                over = elemento[17]      

            if len(elemento) == 21:
                under = elemento[18]
                goals = "".join(re.findall(r'\d+', elemento[17]))
                over = elemento[16]     

            df_bet.loc[len(df_bet.index)]  = [teams, home, tie, away, under, goals, over]
        except:
            pass
    df_bet.to_csv('CSV\df_bet_live.csv')
    return df_bet

In [ ]:
def bet20_live():
    df_20bet = pd.DataFrame(columns=['teams', '1_20b', 'X_20b', '2_20b', 'under_20b', 'goals_20b', 'over_20b'])
    elementos = veinte.find_elements(By.CSS_SELECTOR, f'div[class="event-table__row"]')
    i=0
    for partido in elementos:
        elemento = partido.text.split('\n')
        partido.location_once_scrolled_into_view
        #time.sleep(0.15)
        if len(elemento)==17:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 1
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 1')
        if len(elemento)==18:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 2
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 2')
        if len(elemento)==19:
            teams = elemento[2]+' '+elemento[5]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]        
            tipo  = 3
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 3')
        try:
            df_20bet.loc[len(df_20bet.index)] = [teams, home, tie, away, under, goals, over]
        except:
            pass
            

        

        
    elementos[0].location_once_scrolled_into_view
    df_20bet.drop_duplicates(inplace=True)
    df_20bet.to_csv('CSV\df_20bet_live.csv')
    return df_20bet

In [ ]:
bet20_live()

In [ ]:
def merge():

    '''1XBET // BETWAY '''
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv')
        df_xbet = pd.read_csv('CSV\df_xbet.csv')

        teams_bway = df_bway.teams.tolist()
        
        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_bway, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>55]

        df_xbet_betway = pd.merge(df_xbet, df_bway, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_betway.to_csv('CSV\TwoWay\df_1xbet_betway.csv')
    except:
        print('Something went wrong. Check: 1XBET or BETWAY [block 1] ')
        print(f'1XBET: {df_xbet.shape}')
        print(f'BETWAY: {df_bway.shape}')
        pass

    '''1XBET // BETSSON '''
    try:
        df_bet = pd.read_csv('CSV\df_bet_live.csv')

        teams_bet = df_bet.teams.tolist()

        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>55]

        df_xbet_betsson = pd.merge(df_xbet, df_bet, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_betsson.to_csv('CSV\TwoWay\df_1xbet_betsson.csv')
    except:
        print('Something went wrong. Check: 1XBET or BETSSON [block 2]')
        print(f'1XBET: {df_xbet.shape}')
        print(f'BETSSON: {df_bet.shape} ')
        pass

    ''' 1XBET // 20BET ''' 
    try:
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv')

        teams_20bet = df_20bet.teams.tolist()

        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>55]

        df_xbet_20bet = pd.merge(df_xbet, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_20bet.to_csv('CSV\TwoWay\df_1xbet_20bet.csv')
    except:
        print('Something went wrong. Check: 1xBET or 20BET [block 3]')
        print(f'1XBET: {df_xbet.shape}')
        print(f'20BET: {df_20bet.shape}')
        pass

    ''' 20BET // BETWAY ''' 
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv')

        df_bway[['teams_matched', 'score']] = df_bway.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer = fuzz.token_sort_ratio)).apply(pd.Series)
        df_bway = df_bway[df_bway.score>55]

        df_bway_20bet = pd.merge(df_bway, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_bway_20bet.to_csv('CSV\TwoWay\df_bway_20bet.csv')
    except:
        print('Something went wrong. Check 20BET or BETWAY [block 4]')
        print(f'20BET: {df_20bet.shape}')
        print(f'BETWAY: {df_bway.shape}')

    ''' 20BET // BETSSON '''
    try:
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv')

        teams_20bet = df_20bet.teams.tolist()

        df_bet[['teams_matched', 'score']] = df_bet.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bet = df_bet[df_bet.score>55]

        df_bet_20bet = pd.merge(df_bet, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_bet_20bet.to_csv('CSV\TwoWay\df_betsson_20bet.csv')
    except:
        print('Something went wrong. Check: BETSSON or 20BET [block 5]')
        print(f'BETSSON: {df_bet.shape}')
        print(f'20BET: {df_20bet.shape}')
        pass

    ''' BETWAY // BETSSON '''    
    try:
        teams_bway = df_bway.teams.tolist()
        
        df_bet[['teams_matched', 'score']] = df_bet.teams.apply(lambda x:process.extractOne(x, teams_bway, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bet = df_bet[df_bet.score>55]

        df_bet_betway = pd.merge(df_bet, df_bway, left_on='teams_matched', right_on='teams', how='left')
        df_bet_betway.to_csv('CSV\TwoWay\df_betsson_betway.csv')
    except:
        print('Something went wrong. Check: BETSSON or BETWAY [block 6]')
        print(f'BETSSON: {df_bet.shape}')
        print(f'BETWAY: {df_bway.shape}')
        pass





In [ ]:
merge()

In [ ]:
def final_function(minutos):
    # TIME STUFF
    target = pd.datetime.now() + pd.to_timedelta(minutos, unit='min')
    target_minute = target.minute
    target_hour = target.hour
    now = pd.datetime.now()

    i=0
    #THREADS
    # Put everything to work
    while target>now:

        now = pd.datetime.now()
        
        betway_thread  = threading.Thread(target=betway)
        One_xbet_thread   = threading.Thread(target=One_xbet)
        betsson  = threading.Thread(target=betsson_live)
        bet20 = threading.Thread(target=bet20_live)
        
        clear_output()
        i+=1
        print(f'Will work till: {target_hour}hs and {target_minute}mins')
        print(f'Iteration number: {i}')
        print('-------')
        print('Functions running')

        betway_thread.start()
        One_xbet_thread.start()
        betsson.start()
        bet20.start()

        betway_thread.join()
        One_xbet_thread.join()
        betsson.join()
        bet20.join()
        

        print('Scrapers done')
        print('-------')
        print('Merging start...')
        merge()
        print('-------')
        print('Merging finished! All csv files have been updated')
        time.sleep(5)
    

In [ ]:
final_function(4)